In [2]:
import duckdb  # Importa o DuckDB para manipulação de dados e execução de SQL
import os  # Importa o módulo os para interagir com variáveis de ambiente do sistema
from dotenv import load_dotenv, find_dotenv  # Importa funções para carregar variáveis de ambiente de um arquivo .env
from deltalake import DeltaTable, write_deltalake
# Carrega as variáveis de ambiente definidas no arquivo .env
load_dotenv(find_dotenv())

# Define as credenciais de acesso ao MinIO a partir das variáveis de ambiente
AWS_ACCESS_KEY = os.getenv("AWS_ACCESS_KEY_MINIO")  # Chave de acesso do MinIO
AWS_SECRET_KEY = os.getenv("AWS_SECRET_KEY_MINIO")  # Chave secreta do MinIO
HOST_MINIO = os.getenv("HOST_MINIO")  # Host do MinIO

# Conecta ao DuckDB, criando uma instância de conexão
con = duckdb.connect()

# Cria uma secret no DuckDB para acessar o MinIO com as credenciais fornecidas
con.execute(f"""
    CREATE SECRET my_minio_secret (
        TYPE 'S3',
        KEY_ID '{AWS_ACCESS_KEY}',
        SECRET '{AWS_SECRET_KEY}',
        REGION 'us-east-1',
        ENDPOINT '{HOST_MINIO}:9000',
        URL_STYLE 'path',
        USE_SSL false
    );
""")

# Define o caminho de destino no MinIO para os arquivos Parquet
path_minio_landing = 's3://landing/comex'
path_minio_bronze = 's3://bronze/comex/ncm_sh'

In [3]:
storage_options = {
    "AWS_ENDPOINT_URL": f"http://{HOST_MINIO}:9000",
    "AWS_REGION": "us-east-1",
    "AWS_ACCESS_KEY_ID": AWS_ACCESS_KEY,
    "AWS_SECRET_ACCESS_KEY": AWS_SECRET_KEY,
    "AWS_S3_ALLOW_UNSAFE_RENAME": "true",
    "AWS_ALLOW_HTTP": "true"
}

In [4]:
con.sql(f"""
        SELECT COUNT(DISTINCT CO_SH6) 
         from '{path_minio_landing}/NCM_SH.parquet';        
        """)

┌────────────────────────┐
│ count(DISTINCT CO_SH6) │
│         int64          │
├────────────────────────┤
│                   6609 │
└────────────────────────┘

In [5]:
df = con.sql(f"""
    SELECT 
        CO_SH6 AS cod_sh6,
        NO_SH6_POR AS nome_sh6,
        CO_SH4 AS cod_sh4,
        NO_SH4_POR AS nome_sh4,
        CO_SH2 AS cod_sh2,
        NO_SH2_POR AS nome_sh2,
        CO_NCM_SECROM AS cod_ncm_secrom,
        NO_SEC_POR AS nome_secao
    FROM '{path_minio_landing}/NCM_SH.parquet';
        """).to_arrow_table()

In [6]:
df.to_pandas().head()

,cod_sh6,nome_sh6,cod_sh4,nome_sh4,cod_sh2,nome_sh2,cod_ncm_secrom,nome_secao
0,10110,"Animais vivos das espécies cavalar, asinina e ...",101,"Cavalos, asininos e muares, vivos",1,Animais vivos,I,Animais vivos e produtos do reino animal
1,10111,"Cavalos reprodutores, de raça pura",101,"Cavalos, asininos e muares, vivos",1,Animais vivos,I,Animais vivos e produtos do reino animal
2,10119,"Outros cavalos, vivos",101,"Cavalos, asininos e muares, vivos",1,Animais vivos,I,Animais vivos e produtos do reino animal
3,10120,Asininos e muares vivos,101,"Cavalos, asininos e muares, vivos",1,Animais vivos,I,Animais vivos e produtos do reino animal
4,10121,Cavalos reprodutores de raça pura,101,"Cavalos, asininos e muares, vivos",1,Animais vivos,I,Animais vivos e produtos do reino animal


In [9]:
write_deltalake(
        f'{path_minio_bronze}',
        df,
        storage_options=storage_options
    )

In [7]:
table_path = f'{path_minio_bronze}'

# Conecte à tabela Delta existente
table = DeltaTable(table_path, storage_options=storage_options)



In [8]:
table.merge(
    source=df,
    predicate='target.cod_sh6 = source.cod_sh6',
    source_alias="source",
    target_alias="target",
).when_not_matched_insert_all().execute()


{'num_source_rows': 6609,
 'num_target_rows_inserted': 6582,
 'num_target_rows_updated': 0,
 'num_target_rows_deleted': 0,
 'num_target_rows_copied': 0,
 'num_output_rows': 6582,
 'num_target_files_added': 1,
 'num_target_files_removed': 0,
 'execution_time_ms': 177,
 'scan_time_ms': 0,
 'rewrite_time_ms': 124}

In [9]:
con.sql(f"""
        SELECT *
        FROM delta_scan('{path_minio_bronze}')
        """)

┌─────────┬──────────────────────┬─────────┬───┬─────────────────────┬────────────────┬──────────────────────┐
│ cod_sh6 │       nome_sh6       │ cod_sh4 │ … │      nome_sh2       │ cod_ncm_secrom │      nome_secao      │
│  int64  │       varchar        │  int64  │   │       varchar       │    varchar     │       varchar        │
├─────────┼──────────────────────┼─────────┼───┼─────────────────────┼────────────────┼──────────────────────┤
│   10110 │ Animais vivos das …  │     101 │ … │ Animais vivos       │ I              │ Animais vivos e pr…  │
│   10130 │ Animais da espécie…  │     101 │ … │ Animais vivos       │ I              │ Animais vivos e pr…  │
│   10221 │ Bovinos domésticos…  │     102 │ … │ Animais vivos       │ I              │ Animais vivos e pr…  │
│   10229 │ Outros bovinos dom…  │     102 │ … │ Animais vivos       │ I              │ Animais vivos e pr…  │
│   10310 │ Suínos reprodutore…  │     103 │ … │ Animais vivos       │ I              │ Animais vivos e pr…  │
│

In [10]:
con.close()